In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('Monkey D.Luffy') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/15 14:45:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
!wget "https://noaa-ghcn-pds.s3.amazonaws.com/csv.gz/by_year/2023.csv.gz"

--2023-08-14 16:40:14--  https://noaa-ghcn-pds.s3.amazonaws.com/csv.gz/by_year/2023.csv.gz
Resolving noaa-ghcn-pds.s3.amazonaws.com (noaa-ghcn-pds.s3.amazonaws.com)... 52.216.220.113, 54.231.170.73, 52.217.225.97, ...
Connecting to noaa-ghcn-pds.s3.amazonaws.com (noaa-ghcn-pds.s3.amazonaws.com)|52.216.220.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96782061 (92M) [application/octet-stream]
Saving to: ‘2023.csv.gz’

2023.csv.gz         100%[===================>]  92.30M  2.71MB/s    in 29s     

2023-08-14 16:40:45 (3.20 MB/s) - ‘2023.csv.gz’ saved [96782061/96782061]



In [8]:
import time
from pyspark.sql import functions as F

In [14]:
start_time = time.time()
df = pd.read_csv('2022.csv.gz')
print("--- %s seconds ---" % (time.time() - start_time))

--- 36.61327886581421 seconds ---


In [19]:
df.tail()

,AE000041196,20220101,TAVG,204,H,Unnamed: 5,S,Unnamed: 7
37787619,WF000917530,20221231,TAVG,283,H,NaN,S,NaN
37787620,WQW00041606,20221231,TAVG,278,H,NaN,S,NaN
37787621,WZ004455110,20221231,TAVG,203,H,NaN,S,NaN
37787622,ZI000067775,20221231,TAVG,206,H,NaN,S,NaN
37787623,ZI000067975,20221231,TAVG,202,H,NaN,S,NaN


In [20]:
start_time = time.time()
df_spark = spark.read.csv('2023.csv.gz')
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.3223757743835449 seconds ---


In [3]:
df_spark.head()

NameError: name 'df_spark' is not defined

In [4]:
schema = types.StructType([
    types.StructField("StationId", types.StringType(), True),
    types.StructField("DateStr", types.StringType(), True),
    types.StructField("Element", types.StringType(), True),
    types.StructField("Value", types.DoubleType(), True),
    types.StructField("M-Flag", types.StringType(), True),
    types.StructField("Q-Flag", types.StringType(), True),
    types.StructField("S-Flag", types.StringType(), True),
    types.StructField("Obs-Time", types.StringType(), True)
])

In [5]:
df_2023 = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('2023.csv.gz')

In [12]:
df_2023 = df_2023.withColumn('countryCode', F.substring('StationId', 0, 2))
df_2023.take(5)

NameError: name 'substring' is not defined

In [27]:
output_path = '../data/pq/2023'
df_2023 \
    .write.parquet(output_path, mode='overwrite')

23/08/14 18:12:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: AE000041196, 20230101, TMAX, 252, , , S, 
 Schema: StationId, DateStr, Element, Value, M-Flag, Q-Flag, S-Flag, Obs-Time
Expected: StationId but found: AE000041196
CSV file: file:///Users/admin/Public/project/football-datasets/scripts/2023.csv.gz


In [28]:
year = 2023
color = 'yellow'
month = 1
input_path = f'../data/pq/{year}/part-00000-9ef2930c-6d4f-4430-837b-73b988ff4dcd-c000.snappy.parquet'
dff = spark.read.parquet(input_path)

In [29]:
dff.take(5)

[Row(StationId='AE000041196', DateStr='20230101', Element='TMIN', Value=149.0, M-Flag=None, Q-Flag=None, S-Flag='S', Obs-Time=None),
 Row(StationId='AE000041196', DateStr='20230101', Element='PRCP', Value=0.0, M-Flag='D', Q-Flag=None, S-Flag='S', Obs-Time=None),
 Row(StationId='AE000041196', DateStr='20230101', Element='TAVG', Value=207.0, M-Flag='H', Q-Flag=None, S-Flag='S', Obs-Time=None),
 Row(StationId='AEM00041194', DateStr='20230101', Element='TMAX', Value=255.0, M-Flag=None, Q-Flag=None, S-Flag='S', Obs-Time=None),
 Row(StationId='AEM00041194', DateStr='20230101', Element='TMIN', Value=186.0, M-Flag=None, Q-Flag=None, S-Flag='S', Obs-Time=None)]

In [ ]:
df_2023.write.format()